In [24]:
import os 
import pandas as pd
import subprocess
import glob
import pybedtools as pbt 
pd.set_option('display.max_columns', None)

pbt.set_bedtools_path('/mnt/BioApps/bedtools/bin/')

os.chdir('/mnt/BioHome/jreyna/jreyna/projects/dchallenge/')

genome_sizes = 'results/refs/hg19/hg19.chrom.sizes'

# make the directory to save our data
outdir = 'results/notebooks/Intersect_PieQTLs_with_T1D_Significant_GWAS_SNPs.Test/'
os.makedirs(outdir, exist_ok=True)

In [25]:
gs_fn = 'results/refs/hg19/hg19.chrom.nochr.sizes'
gencode_fn = 'results/refs/gencode/v30/gencode.v30.annotation.bed'
gencode = pd.read_table(gencode_fn, header=None)
gencode = gencode.drop_duplicates(5)
gencode_dict = {k:v for k,v in gencode[[5,6]].values.tolist()} 
gencode_dict.update({v:k for k,v in gencode[[5,6]].values.tolist()})

## Load Significant GWAS

In [26]:
major_gwas = ['T1D_32005708', 'T1D_34594039_GCST90018925', 'T1D_34012112_Gaulton']

In [27]:
data = []
gwas = 'results/main/coloc/Data/T1D_GWAS/*/GRCh37/GWAS_input_colocalization_pval_lt_5eMinus8.txt'
for fn in glob.glob(gwas):
    path_info = fn.split('/')
    df = pd.read_table(fn)
    
    if path_info[5] not in major_gwas:
        continue
        
    df['gwas_source'] = path_info[5]
    data.append(df)    
gwas_df = pd.concat(data)
gwas_df['sid'] = gwas_df['CHR'].str.replace('chr', '') + ':' + gwas_df['POS'].astype(str)

# loading finemap data into bedtools
gwas_bed = gwas_df.iloc[:, [0,1,1]]
gwas_bed.columns = ['chrom', 'start', 'end']
gwas_bed['start'] = gwas_bed['start'] - 1
gwas_pbt = pbt.BedTool.from_dataframe(gwas_bed)

/tmp/ipykernel_62437/915015123.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gwas_bed['start'] = gwas_bed['start'] - 1


In [28]:
individual_gwas_studies = gwas_df.groupby('gwas_source').sid.nunique()
individual_gwas_studies = individual_gwas_studies.to_frame()
individual_gwas_studies.columns = ['Number of Sig. GWAS']

In [29]:
individual_gwas_studies

,Number of Sig. GWAS
gwas_source,
T1D_32005708,21097
T1D_34012112_Gaulton,45798
T1D_34594039_GCST90018925,20419


## Load pieQTLs

In [30]:
pieqtls = glob.glob('results/main/pieqtls/2021_chandra_et_al/*/proximal.pieqtls.tsv')

pieqtl_data = []
for pieqtl in pieqtls:
    print(pieqtl)
    
    cline = pieqtl.split('/')[-2]
    df = pd.read_table(pieqtl)    
    df['cline'] = cline
    pieqtl_data.append(df) 
    
pieqtl_df = pd.concat(pieqtl_data)
pieqtl_df['Chromosome'] = pieqtl_df['Chromosome']
pieqtl_df.loc[:, 'sid'] = pieqtl_df['Chromosome'].str.replace('chr', '') + ':' + \
                          pieqtl_df['pieQTL.Position'].astype(str)

results/main/pieqtls/2021_chandra_et_al/B-cell_naive/proximal.pieqtls.tsv
results/main/pieqtls/2021_chandra_et_al/NK-cell_naive/proximal.pieqtls.tsv
results/main/pieqtls/2021_chandra_et_al/monocyte_naive/proximal.pieqtls.tsv
results/main/pieqtls/2021_chandra_et_al/CD4_T-cell_naive/proximal.pieqtls.tsv
results/main/pieqtls/2021_chandra_et_al/CD8_T-cell_naive/proximal.pieqtls.tsv


In [31]:
individual_cells = pieqtl_df.groupby('cline').Target_geneID.nunique()
individual_cells = individual_cells.to_frame()
individual_cells.columns = ['Number of pieQTL Genes']
individual_cells

,Number of pieQTL Genes
cline,
B-cell_naive,1233
CD4_T-cell_naive,1168
CD8_T-cell_naive,1081
NK-cell_naive,872
monocyte_naive,1065


In [32]:
individual_cells = pieqtl_df.groupby('cline').sid.nunique()
individual_cells = individual_cells.to_frame()
individual_cells.columns = ['Number of pieQTL SNPs']
individual_cells

,Number of pieQTL SNPs
cline,
B-cell_naive,8110
CD4_T-cell_naive,6800
CD8_T-cell_naive,5539
NK-cell_naive,5332
monocyte_naive,6006


In [33]:
pieqtl_bed = pieqtl_df.iloc[:, [1,2,2,-1]]
pieqtl_bed.columns = ['chrom', 'start', 'end', 'cline']
pieqtl_bed.loc[:, 'start'] = pieqtl_bed['start'] - 1
pieqtl_pbt = pbt.BedTool.from_dataframe(pieqtl_bed)

/tmp/ipykernel_62437/2754423286.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pieqtl_bed.loc[:, 'start'] = pieqtl_bed['start'] - 1


In [34]:
pieqtl_bed.shape

(35898, 4)

## Intersect pieQTLs and Sig GWAS

In [35]:
major_cols = ['ge_source',
              'rsid',
             'chrom',
             'pos',
             'geneid',
             'genename',
             'allele1',
             'allele2',
             'maf',
             'beta_x',
             'se',
             'z',
             'prob',
             'log10bf',
             'mean',
             'sd',
             'mean_incl',
             'sd_incl',
             'pval',
             'gwas_source',
             'pieQTL.ID',
             'Chromosome',
             'pieQTL.Position',
             'Target_geneName',
             'TSS',
             'pvalue',
             'FDR (DICE)',
             'beta_y',
             'ref',
             'alt',
             'Mean.TPM.Homozygous.Reference',
             'Mean.TPM.Heterozygous',
             'Mean.TPM.Homozygous.Alternative',
             'Interaction_type',
             'GWAS.Trait',
             'cline',
             'regionID',
             'GWASLoci',
             'index']


### Intersection

In [36]:
gwas_df.head()

,CHR,POS,BETA,SE,P,N,gwas_source,sid
0,chr1,25293941,0.076575,0.013960,4.130000e-08,520580.0,T1D_34012112_Gaulton,1:25293941
1,chr1,25294607,0.077268,0.014040,3.730000e-08,520580.0,T1D_34012112_Gaulton,1:25294607
2,chr1,25294878,0.076429,0.013922,4.030000e-08,520580.0,T1D_34012112_Gaulton,1:25294878
3,chr1,25295580,0.076999,0.013951,3.400000e-08,520580.0,T1D_34012112_Gaulton,1:25295580
4,chr1,25296478,0.077267,0.014047,3.780000e-08,520580.0,T1D_34012112_Gaulton,1:25296478


In [37]:
pieqtl_df.head()

,pieQTL.ID,Chromosome,pieQTL.Position,Target_geneID,Target_geneName,TSS,pvalue,FDR (DICE),beta,ref,alt,Mean.TPM.Homozygous.Reference,Mean.TPM.Heterozygous,Mean.TPM.Homozygous.Alternative,Interaction_type,GWAS.Trait,cline,sid
0,rs9488914,chr6,116690849,ENSG00000111817.12,DSE,116575336,1.340000e-29,5.210000e-24,-1.22,C,T,21.77,12.57,4.70,Direct_pieQTL,,B-cell_naive,6:116690849
1,rs2304748,chr8,33369944,ENSG00000172728.11,FUT10,33330940,1.080000e-27,2.200000e-22,-1.26,T,C,5.24,3.01,1.16,Direct_pieQTL,,B-cell_naive,8:33369944
2,rs2581897,chr8,33371146,ENSG00000172728.11,FUT10,33330940,1.080000e-27,2.200000e-22,-1.26,T,G,5.24,3.01,1.16,Direct_pieQTL,,B-cell_naive,8:33371146
3,rs2581899,chr8,33371199,ENSG00000172728.11,FUT10,33330940,1.080000e-27,2.200000e-22,-1.26,T,C,5.24,3.01,1.16,Direct_pieQTL,,B-cell_naive,8:33371199
4,rs2732288,chr8,33370757,ENSG00000172728.11,FUT10,33330940,1.080000e-27,2.200000e-22,-1.26,G,A,5.24,3.01,1.16,Direct_pieQTL,,B-cell_naive,8:33370757


In [38]:
m = gwas_df.merge(pieqtl_df, left_on=['CHR', 'POS'], right_on=['Chromosome', 'pieQTL.Position'])

In [39]:
len(m.Target_geneID.unique())

91